In [79]:
%load_ext autoreload
%autoreload 2

In [80]:
import networkx as nx
from visualize import Tabularizer
%matplotlib inline

In [174]:
g = nx.MultiDiGraph()
with open('data/reactome.cif') as f:
    for line in f:
        e1, e2, r = line.strip().split('\t')
        g.add_edge(e1, e2, label=r)
        
with open('data/reactome_references.json') as f:
    references = json.load(f)

In [175]:
tabularizer = Tabularizer(g, references)

In [176]:
reactions = tabularizer.get_reactions()

In [177]:
reactions_by_participants = defaultdict(list)
for r in reactions:
    reactions_by_participants[frozenset(r['participants'])].append(r)

In [178]:
import pandas as pd
query_graph = pd.read_csv('data/nfkappab.tsv', sep='\t')

import itertools
proteins = set()
for complx in itertools.chain(query_graph['Protein A'], query_graph['Protein B']):
    proteins.update(complx.split(':'))


interesting_reactions = [r for r in reactions if len(set(r['participants']) & set(proteins)) > 0]

In [228]:
interesting_reactions[11]

{'lefts': ['CHUK:IKBKG:IKBKB', ''],
 'rights': ['IKBKG:IKBKB:CHUK', ''],
 'participants': ['O15111',
  'Q9Y6K9',
  'O14920',
  'Q9Y6K9',
  'O15111',
  'O14920',
  'O43318',
  'Q15750',
  'Q9Y4K3',
  'Q04759'],
 'id': 'http://identifiers.org/reactome/R-HSA-2730876',
 'inhibitors': [],
 'regulators': [],
 'activators': ['TAB1:http://pathwaycommons.org/pc2/#Protein_a2f1e5ef306b12349b07b7454e64d398:MAP3K7:TRAF6:PRKCQ'],
 'references': ['http://identifiers.org/pubmed/17047224',
  'http://identifiers.org/pubmed/20300203',
  'http://identifiers.org/pubmed/11325957',
  'http://identifiers.org/pubmed/14514672',
  'http://identifiers.org/pubmed/11460167']}

In [229]:
len(reactions)

11486

In [ ]:
import json

with open('./uniprot_to_hgnc.json') as f:
    uniprot_to_hgnc = json.load(f)

def get_type(node):
    if '#' not in node:
        return 'Protein'
    else:
        return node.split('#')[1].split('_')[0]
node_indices = {n: i for i, n in enumerate(toy_g.nodes())}
nodes = [{'name': uniprot_to_hgnc.get(n, n), 'type': get_type(n)} for n in toy_g.nodes()]
links = [{'source': node_indices[e[0]], 'target': node_indices[e[1]], 'type': e[2]['label']} for e in toy_g.edges(data=True)]

with open('graph.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links}, f, indent=4)